In [ ]:
import os
hostname = os.popen("hostname").read().split("\n")[0]
if(hostname != "reckoner1429-Predator-PH315-52" and hostname != "jarvis"):
    from google.colab import drive
    from google.colab import drive
    drive.mount('/content/gdrive')

    ! chmod 755 "/content/gdrive/My Drive/collab-var.sh"
    ! "/content/gdrive/My Drive/collab-var.sh"
    %cd "/content/gdrive/My Drive/github/video-emotion-recognition"

In [ ]:
import librosa
import librosa.display
import numpy as np
import utils.data_util as data_util
import utils.preprocess_util as preprocess_util
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Flatten, Concatenate
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import load_model
import tensorflow.keras as keras
import time
import utils.config as config
from utils.hyparam_util import load_fusion_hyparam
import sklearn.metrics as skm


In [ ]:
# if(config.CURRENT_DATASET == 'SAVEE'):
#     dataset = preprocess_util.SAVEE()
# elif(config.CURRENT_DATASET == 'RAVDESS'):
#     dataset = preprocess_util.RAVDESS()
dataset = preprocess_util.RML()
SEED = 0 
X_train_audio, X_test_audio, Y_train_audio, Y_test_audio = dataset.load_audio_filenames(SEED, 0.2)
X_train_face, X_test_face, Y_train_face, Y_test_face = dataset.load_visual_filenames(SEED, 0.2)
print(X_test_audio)
print(Y_test_audio)

In [ ]:
iteration = "A3"
path = os.path.join(dataset.MODEL_SAVE_DIR, "iteration-"+iteration)
new_model = tf.keras.models.load_model(path +'/saved_models/xception-8-8-8-A3.h5')

#new_model.summary()


In [ ]:
INPUT_WIDTH = 224
INPUT_HEIGHT = 224
hyparams = load_fusion_hyparam(iteration)
BATCH_SIZE = hyparams['batch_size']
N_CLASSES = len(dataset.emotion_classes)
epochs = hyparams['epochs']
# ITERATION = hyparams['iteration']

In [ ]:
X_val_gen = data_util.MultimodalDataGenerator(X_test_face, X_test_audio, Y_test_face, BATCH_SIZE, INPUT_WIDTH, INPUT_HEIGHT)

In [ ]:
pred = new_model.predict(X_val_gen)

In [ ]:
#confusion matrix
print(type(pred))
print(pred.shape)
print(Y_test_face.shape)

EMOTION_CLASSES = dataset.emotion_classes

true_values = []
pred_values = []

for i in range(len(pred)):
    true_values.append(EMOTION_CLASSES[np.argmax(Y_test_face[i])])
    pred_values.append(EMOTION_CLASSES[np.argmax(pred[i])])
    
print(true_values)
print(pred_values)
            

cm = skm.confusion_matrix(true_values, pred_values, labels = EMOTION_CLASSES)

print(cm)

fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm)
plt.title('Confusion matrix of the densenet121\n')
c=fig.colorbar(cax)
# c.add_lines(CS, erase=True)
#ax.set_xticklabels([' '] + EMOTION_CLASSES)
#ax.set_yticklabels([' '] + EMOTION_CLASSES)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [ ]:
#Histories
path = dataset.MODEL_SAVE_DIR
path_fusion = path + '/iteration-A3/history'
path_ftm = path + '/ftm-0/history'
# print(path_ftm)
# print(path_fusion)
model_histories = {}

for model_history in os.listdir(path_fusion):
    model_history_path = path_fusion + '/' + model_history
    if(os.path.isfile(model_history_path)):
        model_histories[model_history.split('.')[0]] = np.load(model_history_path, allow_pickle = True)
        print(model_history)
#print("=======================================================================================")
        
for model_history in os.listdir(path_ftm):
    model_history_path = path_ftm + '/' + model_history
    if(os.path.isfile(model_history_path)):
        model_histories[model_history.split('.')[0]] = np.load(model_history_path, allow_pickle = True)
        print(model_history)
#print("=======================================================================================")

#change model name here to check history
model_name = "densenet121"
key_fusion = model_name + "-8-8-8-A3-history"
key_face = "ftm-" + model_name + "-face-8-history"
key_audio = "ftm-" + model_name + "-audio-8-history"

print(model_histories[key_face])
print("=======================================================================================")

print(model_histories[key_audio])
print("=======================================================================================")

print(model_histories[key_fusion])
print("=======================================================================================")
